In [0]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle, gzip
import copy
import math
from torchvision import datasets, transforms
from PIL import ImageOps
from torch.utils.data import ConcatDataset
import torch.utils.data as data_utils
from torch.autograd import Variable
from tqdm import tqdm
from IPython import display
from copy import deepcopy

In [0]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
code_size= 100

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Permuted MNIST

In [0]:
class PermutedMnistGenerator():
    def __init__(self, max_iter=10, random_seed=0):
        # Open data file
        f = gzip.open('/content/drive/My Drive/Jupyter/BayesML_Burnaev/CL_project/mnist.pkl.gz', 'rb')
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
        f.close()

        # Define train and test data
        self.X_train = np.vstack((train_set[0], valid_set[0]))
        self.Y_train = np.hstack((train_set[1], valid_set[1]))
        self.X_test = test_set[0]
        self.Y_test = test_set[1]
        self.random_seed = random_seed
        self.max_iter = max_iter
        self.cur_iter = 0

        self.out_dim = 10           # Total number of unique classes
        self.class_list = range(10) # List of unique classes being considered, in the order they appear

        # self.classes is the classes (with correct indices for training/testing) of interest at each task_id
        self.classes = []
        for iter in range(self.max_iter):
            self.classes.append(range(0,10))

        self.sets = self.classes

    def get_dims(self):
        # Get data input and output dimensions
        return self.X_train.shape[1], self.out_dim

    def next_task(self):
        if self.cur_iter >= self.max_iter:
            raise Exception('Number of tasks exceeded!')
        else:
            np.random.seed(self.cur_iter+self.random_seed)
            perm_inds = list(range(self.X_train.shape[1]))
            # First task is (unpermuted) MNIST, subsequent tasks are random permutations of pixels
            if self.cur_iter > 0:
                np.random.shuffle(perm_inds)

            # Retrieve train data
            next_x_train = deepcopy(self.X_train)
            next_x_train = next_x_train[:,perm_inds]

            # Initialise next_y_train to zeros, then change relevant entries to ones, and then stack
            next_y_train = np.zeros((len(next_x_train), 10))
            next_y_train[:,0:10] = np.eye(10)[self.Y_train]

            # Retrieve test data
            next_x_test = deepcopy(self.X_test)
            next_x_test = next_x_test[:,perm_inds]

            next_y_test = np.zeros((len(next_x_test), 10))
            next_y_test[:,0:10] = np.eye(10)[self.Y_test]

            self.cur_iter += 1

            return next_x_train, next_y_train, next_x_test, next_y_test

    def reset(self):
        self.cur_iter = 0

# GAN

In [0]:
class GAN_generator(nn.Module):
    def __init__(self, code_size, output_size):
        super(GAN_generator, self).__init__()
        #n_features = 100
        #n_out = 784
        self.generator_net =  nn.Sequential(
            nn.Linear(code_size, 256),
            nn.LeakyReLU(0.2),            
            nn.Linear(256, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, output_size)
        )

    def forward(self, x):
        return self.generator_net(x.cuda())
    
    
class GAN_discriminator(nn.Module):
    def __init__(self, input_size, output_size):
        super(GAN_discriminator, self).__init__()
        #n_features = 784
        #n_out = 1
        self.discriminator_net = nn.Sequential( 
            nn.Linear(input_size, 1024),
            nn.LeakyReLU(0.2),
            nn.Linear(1024, 512),
            nn.LeakyReLU(0.2),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, output_size)
        )    

    def forward(self, x):
        return self.discriminator_net(x.cuda())

    
# Noise
def sample_noise_batch(batch_size, code_size):
    n = Variable(torch.randn(batch_size, code_size))
    return n.cuda() 
    
def images_to_vectors(images):
    return images.view(images.size(0), 784)

def vectors_to_images(vectors):
    return vectors.view(vectors.size(0), 1, 28, 28) 

def generator_loss(generator, discriminator, noise):
    generated_data = generator(noise)
    disc_on_generated_data = discriminator(generated_data)
    logp_gen_is_real = F.logsigmoid(disc_on_generated_data)
    loss = -torch.mean(logp_gen_is_real, 0)
    return loss

def discriminator_loss(discriminator, real_data, generated_data):
    disc_on_real_data = discriminator(real_data)
    disc_on_fake_data = discriminator(generated_data)
    logp_real_is_real = F.logsigmoid(disc_on_real_data)
    logp_gen_is_fake = F.logsigmoid(- disc_on_fake_data)
    loss = -torch.mean(logp_real_is_real, 0) - torch.mean(logp_gen_is_fake, 0)
    return loss

In [0]:
def sample_images(axs, generator, nrow, ncol, code_size, sharp=False):
    images = generator(sample_noise_batch(batch_size=nrow*ncol, code_size=code_size))
    images = images.data.cpu().numpy().reshape(nrow*ncol, 28, 28)
    if np.var(images)!=0:
        images = images.clip(0., 1.)#np.min(train_set[0]),np.max(train_set[0]))
    for i in range(nrow*ncol):
        ax = axs[i // ncol][i % ncol]
        if sharp:
            ax.imshow(images[i], cmap="gray", interpolation="none")
        else:
            ax.imshow(images[i], cmap="gray")
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        
    '''
        plt.subplot(nrow,ncol,i+1)
        if sharp:
            plt.imshow(images[i], interpolation="none")
        else:
            plt.imshow(images[i])
        
    plt.show()
    '''
    return 
def sample_probas(ax, discriminator, generator, batch_size, x_batch, code_size):
    plt.title('Generated vs real data')
    
    D_real = torch.sigmoid(discriminator(x_batch))
    generated_data_batch = generator(sample_noise_batch(batch_size, code_size))
    D_fake = torch.sigmoid(discriminator(generated_data_batch))
    
    ax.hist(D_real.data.cpu().numpy(),
             label='D(x)', alpha=0.5, range=[0,1])
    ax.hist(D_fake.data.cpu().numpy(),
             label='D(G(z))', alpha=0.5, range=[0,1])
    ax.legend(loc='best')
    #plt.show()


In [0]:
def GAN_train(discr_input, discr_output, gen_input, gen_output, batch_size, data_loader, n_epochs):
    d_losses = []
    g_losses = []
    max_g = 3
    axs = [[[] for _ in range(max_g)] for _ in range(2)]
    
    discriminator = GAN_discriminator(discr_input, discr_output)
    discriminator.cuda()
    generator = GAN_generator(gen_input, gen_output)
    generator.cuda()
    
    #optimizers
    disc_opt = torch.optim.Adam(discriminator.parameters(), lr=2e-4, betas=[0.5, 0.999])
    gen_opt = torch.optim.Adam(generator.parameters(), lr=2e-4, betas=[0.5, 0.999])
    
    for epoch in range(n_epochs):
        if  len(g_losses) == 0 or (g_losses[-1] < d_losses[-1] * 16): #Hack? Yeah, it's hack.
            d_loss = []
            for i in range(10):
                # Train discriminator
                data_loader_iter = iter(data_loader)
                x_batch, y_batch = next(data_loader_iter)
                fake_data = generator(sample_noise_batch(batch_size, gen_input)) #gen_input==code_size
                disc_loss = discriminator_loss(discriminator, x_batch, fake_data)
                disc_opt.zero_grad()
                disc_loss.backward()
                disc_opt.step()
                d_loss.append(disc_loss.data.cpu().numpy()[0])

                
        if len(d_losses) > 0:
            d_losses.append(np.mean(d_loss) * 0.05 + d_losses[-1] * (1 - 0.05))
        else:
            d_losses.append(np.mean(d_loss))

        # Train generator
        noise = sample_noise_batch(batch_size, gen_input)
        gen_loss = generator_loss(generator, discriminator, noise)
        gen_opt.zero_grad()
        gen_loss.backward()
        gen_opt.step()
        
        if len(g_losses) > 0:
            g_losses.append(gen_loss.data.cpu().numpy()[0] * 0.05 + g_losses[-1] * (1 - 0.05))
        else:
            g_losses.append(gen_loss.data.cpu().numpy()[0])
    
        
        #plot results
        if epoch %100==0:
           
            #display.clear_output(True)
            '''
            plt.figure(figsize=(18, 4))

            ax = plt.subplot2grid((2,3 + max_g + 2), (0,0), colspan=3, rowspan=2)
            ax.set_title("D-Loss")
            ax.set_xlabel("#iteration")
            ax.set_ylabel("loss")
            ax.plot(g_losses, 'r', label='G-loss', alpha=0.8)
            ax.plot(d_losses, 'b', label='D-loss', alpha=0.8)
            ax.legend()
            ax.grid(True)

            for i in range(0, 2):
                for j in range(3, 3 + max_g):
                    axs[i][j - 3] = plt.subplot2grid((2, 3 + max_g + 2), (i, j))
            sample_images(axs, generator, 2, max_g, gen_input, True)

            ax = plt.subplot2grid((2, 3 + max_g + 2), (0, 3 + max_g), colspan=2, rowspan=2)
            prev_hist = sample_probas(ax, discriminator, generator, 1000, x_batch, gen_input)
            plt.tight_layout()
            plt.show()

            
            sample_images(generator, 2, 3, gen_input, sharp=True)
            sample_probas(discriminator, generator, 1000, x_batch, gen_input)
            '''
            #print('epoch {}\n'.format(epoch))
            #print('dlosses {}\n'.format(d_losses))
            #print('glosses {}\n'.format(g_losses))

    return generator

# Solver

In [0]:
class BayesLinear(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        
        # Weight parameters 1-dim params for each layer
        self.weight_mu = nn.Parameter(torch.zeros(out_features, in_features))
        self.weight_sigma = nn.Parameter(1e-6 * torch.ones(out_features, in_features))
        self.weight = torch.distributions.Normal(self.weight_mu, self.weight_sigma)
        
        # Prior distribution
        self.weight_prior_mu = torch.Tensor([0.]).to(device) #torch.zeros(in_features)
        self.weight_prior_sigma = torch.Tensor([1.]).to(device) #torch.ones(in_features)
        #self.weight_prior = torch.distributions.Normal(self.weight_prior_mu, self.weight_prior_sigma)
        
        self.log_prior = 0.
        self.log_variational_posterior = 0.

    def forward(self, input, sampling=False, calculate_log_probs=False):
        if self.training or sampling:
            weight = self.weight.sample()
        else:
            weight = self.weight.mean
        '''
        if self.training or calculate_log_probs:
            self.log_prior = self.weight_prior.log_prob(weight)
            self.log_variational_posterior = self.weight.log_prob(weight)
        else:
            self.log_prior, self.log_variational_posterior = 0., 0.
        '''
        return F.linear(input, weight)
    
    def kl(self):
        return (torch.log(torch.sqrt(self.weight_prior_sigma)) - torch.log(torch.sqrt(self.weight_sigma)) + (
            (self.weight_mu - self.weight_prior_mu) ** 2 + self.weight_sigma - self.weight_prior_sigma) / (
            2. * self.weight_prior_sigma)).sum()
    

In [0]:
class Solver(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        #hidden_size=100
        super(Solver, self).__init__()
        self.l1 = BayesLinear(input_size, hidden_size)
        self.l2 = BayesLinear(hidden_size, hidden_size)
        self.l3 = BayesLinear(hidden_size, output_size)
        self.relu = nn.ReLU()
        
    def forward(self, x, sampling=False):
        output = self.l1(x, sampling=sampling)
        output = self.relu(output)
        output = self.l2(output, sampling=sampling)
        output = self.relu(output)
        return self.l3(output, sampling=sampling)
    
    
    def log_prior(self):
        return self.l1.log_prior \
               + self.l2.log_prior \
               + self.l3.log_prior
    
    def log_variational_posterior(self):
        return self.l1.log_variational_posterior \
               + self.l2.log_variational_posterior \
               + self.l2.log_variational_posterior
    
    
    def loss(self, gans, solvers, x_curr, y_curr):
        kl = self.l1.kl() + self.l2.kl() + self.l3.kl()
        sumloglik = 0.
        if len(gans) > 0:
            for gan, solver in zip(gans[:-1], solvers[:-1]):
                x_gan = gan(sample_noise_batch(batch_size=len(x_curr), code_size=code_size))
                y_gan = solver(x_gan, sampling=True)
                logits = self.forward(x_gan, sampling=True)
                sumloglik += F.binary_cross_entropy_with_logits(logits, y_gan)
            x_gan_new = gans[-1](sample_noise_batch(batch_size=60000, code_size=code_size))
            y_gan_new = solvers[-1](x_gan_new, sampling=True)
            logits = self.forward(torch.cat([x_gan_new, x_curr],dim=0), sampling=False)
            sumloglik += F.binary_cross_entropy_with_logits(logits, torch.cat([y_gan_new, y_curr],dim=0)) 
                
        else:
            logits = self.forward(x_curr, sampling=True)
            sumloglik = F.binary_cross_entropy_with_logits(logits, y_curr)
            
        return kl + sumloglik #actually negsumloglik

In [0]:
def solver_train_predict(input_size, hidden_size, output_size, batch_size, train_loader, test_loader, n_epochs, seen_tasks,
                         gans, solvers):
    state_dict_path = '/content/drive/My Drive/Jupyter/BayesML_Burnaev/CL_project/solver_weights.pth'
    net = Solver(input_size, hidden_size, output_size)
    if len(gans) > 0:
        net.load_state_dict(torch.load(state_dict_path))
    net.to(device)
    optimizer = optim.Adam(net.parameters())
    
    #training
    net.train()
    loss_list = []
    for epoch in tqdm(range(n_epochs)):
        cumulative_loss = 0.
    
        for i, (x_train, y_train) in enumerate(train_loader):
            x_train = x_train.to(device)
            y_train = y_train.to(device)
            
            net.zero_grad()
            loss = net.loss(gans, solvers, x_train, y_train)
            loss.backward()
            optimizer.step()
            
            cumulative_loss += loss.detach().cpu().numpy()
       
        loss_list.append(cumulative_loss / float(i + 1))
        print('solver_losses = {}\n'.format(loss_list))
        torch.save(net.state_dict(), state_dict_path)
    #prediction
    net.eval()
    with torch.no_grad():
        preds_acc_sum = 0.
        for i, (x_test, y_test) in enumerate(test_loader):
            x_test = x_test.to(device)
            y_test = y_test.to(device)
            logits = net(x_test, sampling=True)
            pred_idx = logits.max(1, keepdims=True)[1].transpose(0,1).detach().cpu().numpy()[0]
            y_test_np = y_test.detach().cpu().numpy()
            y_test_idx = [np.where(r==1)[0][0] for r in y_test_np]
            preds_acc_sum += float(np.sum(pred_idx == y_test_idx) / len(pred_idx))
        mean_preds_acc = preds_acc_sum / float(i + 1)
        
    return net, mean_preds_acc, loss_list

# Main loop

In [0]:
def scholar_train(n_tasks=5, batch_size=256, gan_epochs=301, solver_epochs=5):
    gans = []
    solvers = []
    solver_losses = []
    data_gen = PermutedMnistGenerator(max_iter=n_tasks, random_seed=0)
    print('\n dataset generated, starting tasks')
    pred_accs = []
    for task in tqdm(range(n_tasks)):
        
        #load task data
        x_new_train, y_new_train, x_new_test, y_new_test = data_gen.next_task()
        
        x_train = torch.Tensor(x_new_train).to(device)
        y_train = torch.Tensor(y_new_train).to(device)
        train = data_utils.TensorDataset(x_train, y_train)
        train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=True)
        
        x_test = torch.Tensor(x_new_test).to(device)
        y_test = torch.Tensor(y_new_test).to(device)
        test = data_utils.TensorDataset(x_test, y_test)
        test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=True)
        
        
        print('task {} data loaded'.format(task))
        if task !=0:
            
            #train curr task Solver
            print('running solver task {}'.format(task))
            curr_solver, mean_pred_acc, curr_loss_list = solver_train_predict(input_size=784, hidden_size=100, output_size=10, batch_size=batch_size, 
                             train_loader=train_loader, test_loader=test_loader, n_epochs=solver_epochs, seen_tasks=task, 
                                               gans=gans, solvers=solvers)
            solvers.append(curr_solver)
            pred_accs.append(mean_pred_acc)
            solver_losses.append(curr_loss_list)
            
            #train curr task GAN
            print('running GAN task {}'.format(task))
            curr_generator = GAN_train(discr_input=784, discr_output=1, gen_input=code_size, gen_output=784, 
                                   batch_size=batch_size, data_loader=train_loader, n_epochs=gan_epochs)
            gans.append(curr_generator)
            
            
        else:
            print('running task {}'.format(task))
            
            #train first Solver
            print('running solver task {}'.format(task))
            curr_solver, mean_pred_acc, curr_loss_list = solver_train_predict(input_size=784, hidden_size=100, output_size=10, batch_size=batch_size, 
                             train_loader=train_loader, test_loader=test_loader, n_epochs=solver_epochs, seen_tasks=task, 
                                               gans=gans, solvers=solvers)
            solvers.append(curr_solver)
            pred_accs.append(mean_pred_acc)
            solver_losses.append(curr_loss_list)
            
            #train first GAN
            print('running GAN task {}'.format(task))
            curr_generator = GAN_train(discr_input=784, discr_output=1, gen_input=code_size, gen_output=784, 
                                   batch_size=batch_size, data_loader=train_loader, n_epochs=gan_epochs)
            gans.append(curr_generator)
        
        display.clear_output(True)
        print('pred_accs {}'.format(pred_accs))
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.title('Prediction accuracy')
        plt.xlabel('n_tasks')
        plt.ylabel('accuracy')
        plt.plot(pred_accs, 'b')
        
        plt.subplot(1, 2, 2)
        plt.title('Task {} loss'.format(task))
        plt.xlabel('n_iter')
        plt.ylabel('loss')
        plt.plot(curr_loss_list, 'r')
        
        plt.savefig('/content/drive/My Drive/Jupyter/BayesML_Burnaev/CL_project/acc_plot{}_tasks.png'.format(task))
        
        
        plt.show()
        
        #print('gans after task {}: {}'.format(task, gans))
        #print('solvers after task {}: {}'.format(task, solvers))

        

In [0]:
from google.colab import drive
drive.mount('/content/drive')
scholar_train(n_tasks=10, batch_size=256, gan_epochs=301, solver_epochs=100)#301, 120

Mounted at /content/drive


















  0%|          | 0/10 [00:00<?, ?it/s]


 dataset generated, starting tasks



















  0%|          | 0/100 [00:00<?, ?it/s]

task 0 data loaded
running task 0
running solver task 0


...

solver_losses = [186728.3824468085, 178991.9180851064, 174038.49574468084, 168876.46482712767, 163819.36429521276, 158982.1829787234, 154393.94295212766, 150047.95212765958, 145924.0662898936, 141998.38184840424, 138247.38843085108, 134649.52898936172, 131185.74265292555, 127839.43670212766, 124596.24906914894, 121443.85388962766, 118371.6652925532, 115370.6157912234, 112432.89039228723, 109551.80056515957, 106721.62692819149, 103937.46898271277, 101195.08553856383, 98490.91419547872]




















 25%|██▌       | 25/100 [00:39<01:57,  1.57s/it]

solver_losses = [186728.3824468085, 178991.9180851064, 174038.49574468084, 168876.46482712767, 163819.36429521276, 158982.1829787234, 154393.94295212766, 150047.95212765958, 145924.0662898936, 141998.38184840424, 138247.38843085108, 134649.52898936172, 131185.74265292555, 127839.43670212766, 124596.24906914894, 121443.85388962766, 118371.6652925532, 115370.6157912234, 112432.89039228723, 109551.80056515957, 106721.62692819149, 103937.46898271277, 101195.08553856383, 98490.91419547872, 95821.87230718085]




















 26%|██▌       | 26/100 [00:40<01:55,  1.56s/it]

solver_losses = [186728.3824468085, 178991.9180851064, 174038.49574468084, 168876.46482712767, 163819.36429521276, 158982.1829787234, 154393.94295212766, 150047.95212765958, 145924.0662898936, 141998.38184840424, 138247.38843085108, 134649.52898936172, 131185.74265292555, 127839.43670212766, 124596.24906914894, 121443.85388962766, 118371.6652925532, 115370.6157912234, 112432.89039228723, 109551.80056515957, 106721.62692819149, 103937.46898271277, 101195.08553856383, 98490.91419547872, 95821.87230718085, 93185.40036569149]

